In [ ]:
## 使用Qwen-coder，对保险客户数据表进行SQL查询
"""
 自己编写,本质是：LLM + RAG
    选择适合的LLM，比如：ChatModel: DeepSeek-V3，CodeModel: Qwen2.5-Coder, CodeGeeX2-6B
    RAG，可以分成：向量数据库检索 + 固定文件（比如本地数据表说明等）
    优点：重心在于RAG的提供上，准确性高，配置灵活
    不足：需要设置的条件规则多
"""

In [ ]:
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role
import time
import pandas as pd
import re
# 从环境变量获取 dashscope 的 API Key
os.environ['DASHSCOPE_API_KEY'] = 'sk-58f051ae745e4bb19fdca31735105b11'
api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = api_key

# 封装模型响应函数
def get_response(messages):
    response = dashscope.Generation.call(
        model='qwen-coder-plus',
        messages=messages,
        result_format='message'  # 将输出设置为message形式
    )
    return response

# 从模型响应中提取SQL代码
def get_sql_code(response):
    # 查找```sql和```之间的内容
    pattern = r'```sql(.*?)```'
    # 使用re.search在response内容中搜索SQL代码块
    # pattern: 匹配模式 - ```sql和```之间的内容
    # response.output.choices[0].message.content: 模型返回的完整响应文本
    # re.DOTALL: 允许.匹配任何字符，包括换行符
    match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # 如果没有找到```sql标记，尝试查找任何```之间的内容
        pattern = r'```(.*?)```'
        match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            # 如果没有找到任何代码块，返回整个响应
            return response.output.choices[0].message.content

# 得到sql
def get_sql(query,create_sql):
    start_time = time.time()
    # TODO 方式三的重要写法，开头和结尾写上sql
    sys_prompt = """我正在编写SQL，以下是数据库中的数据表和字段，请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL，如果有多个查询语句，请尝试合并为一个。编写SQL请采用```sql
    """
    user_prompt = f"""-- language: SQL
### Question: {query}
### Input: {create_sql}
### Response:
Here is the SQL query I have generated to answer the question `{query}`:
```sql
"""

    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = get_response(messages)
    return response

######## 需要人工设置 ########
save_file = f'./QA/sql_result_qwen_coder.xlsx'  # 评测结果保存文件
qa_file = './QA/qa_list-2.txt'                  # QA测试题
sql_file = './data/create_sql.txt'           # SQL数据表

In [ ]:
# 读取 SQL数据表
with open(sql_file, 'r', encoding='utf-8') as file:
    create_sql = file.read()
# 读取 SQL问题列表
with open(qa_file, 'r', encoding='utf-8') as file:
    qa_list = file.read()
qa_list = qa_list.split('=====')

# 保存SQL结果
sql_list = []
markdown_list = []
time_list = []
for qa in qa_list:
    query = qa
    query = query.replace('\n', '')
    print(query)
    start_time = time.time()
    # 请求生成sql
    #content, prompt_len = get_sql(query)
    response = get_sql(query,create_sql)
    use_time = round(time.time()-start_time, 2)
    time_list.append(use_time)
    print('SQL生成时间：', use_time)
    print('response=', response.output.choices[0].message.content)
    # 提取生成的SQL
    sql = get_sql_code(response)
    print('SQL: {}'.format(sql))
    sql_list.append(sql)

In [ ]:
# 创建一个空的DataFrame对象，用于存储查询结果
# columns参数指定了三列:
# - QA: 存储问答内容
# - SQL: 存储SQL查询语句
# - time: 存储查询执行时间
result = pd.DataFrame(columns=['QA', 'SQL', 'time'])
result['QA'] = qa_list
result['SQL'] = sql_list
result['time'] = time_list
result.to_excel(save_file, index=False)
result